# Prediction de cas de Covid 19 en Canada

- Felipe Gonzalez

Le but de cette projet est de predire le numero de cas des personnes contagiées par jour dans des provinces de Canada.

Le projet a 3 notebooks:

1. Collecte et integration de données
2. Traitement
3. Developpement du modele

## 1. Collecte et Integration de données

Dans cette notebook, on va faire la collecte et integration de données en considerant les 3 sources de données suivants:

1. Information des Covid-19 au Canada
2. Climat
3. Population

## Collecte de données

Pour la collecte de données du covid19 et le climat, on va prendre les fichiers sur les urls oficielles du governement du Canada et on va les stocker dans le cluster DBFS Databricks.

Utilisation:
  
- Si c'est la premiere fois que vous allez demarer cette notebook, vous devez choisir toutes les variables suivantes comme True
- Si vous voulez avoir mettre à jour les données covid-19, vous devez aussi mettre à jour les données du climat 2021

In [0]:
# Metre a jour des données covid19:
update_covidDB = True
# Metre a jour des données climat 2021:
update_2021weatherdata = True
# Metre a jour des données climat 2020:
update_2020weatherdata = False

La classe suivante gère la base de données obtenue à partir d'une direction URL: Télécharger, Enregistrer, Lire

In [0]:
import urllib.request

# Class to handle downloading data from url and storing in dbfs
class Handle_datafiles:
  def __init__(self, url, path, file):
    self.url = url
    self.path = path
    self.file = file
    
  # save file from url to dbfs
  def SaveFile_FromUrl(self):
    urllib.request.urlretrieve(self.url,"/tmp/"+self.file)
    dbutils.fs.mv("file:/tmp/"+self.file, self.path + self.file)

  # funtion to read databases from dbfs depending on number of lines we want (important for 2021 weather data)
  def ReadFile(self, n = None):
    if n is None:
        df_tmp = spark.read.csv(self.path + self.file, header=True, inferSchema= True)
    else:
        df_tmp = spark.read.csv(self.path + self.file, header=True, inferSchema= True).limit(n) # number of lines read
    return df_tmp

  # function to save in dbfs and read depending on a update conditional
  def SaveAndRead(self, update, n = None):
    if(update):
      self.SaveFile_FromUrl()
    df = self.ReadFile(n)    
    return df
  
  # save from url and read from dbfs
  def SaveAndReadFromUrl(self, n = None):
    self.SaveFile_FromUrl()
    df = self.ReadFile(n)    
    return df

### 1. Information des Covid-19 au Canada

In [0]:
# Take data from url, store it into DBFS, and read it.
url_covidDB = "https://health-infobase.canada.ca/src/data/covidLive/covid19-download.csv"
path_covidDB = "/FileStore/tables/Covid_DB_08Mar/"
file_covidDB = "covid_data.csv"

df_covid_data = Handle_datafiles(url_covidDB, path_covidDB, file_covidDB).SaveAndRead(update_covidDB)

In [0]:
# Remove Canada and Repatriated travellers data
df_covid = df_covid_data.filter((df_covid_data.prname != 'Canada') & (df_covid_data.prname != 'Repatriated travellers'))

In [0]:
Provinces_tmp = [x.prname for x in df_covid.select("prname").distinct().collect()]
print(Provinces_tmp)

['Manitoba', 'Yukon', 'Nova Scotia', 'Northwest Territories', 'Newfoundland and Labrador', 'Alberta', 'Nunavut', 'New Brunswick', 'Saskatchewan', 'Prince Edward Island', 'Ontario', 'British Columbia', 'Quebec']

In [0]:
# Check the last day of data
from pyspark.sql import SQLContext

df_covid.createOrReplaceTempView("table_prediction")
query_latest_rec = """SELECT date,prname,numtoday FROM table_prediction WHERE prname = 'Quebec' ORDER BY date DESC limit 1"""
latest_rec = sqlContext.sql(query_latest_rec)
latest_rec.show()

+----------+------+--------+
 date|prname|numtoday|
+----------+------+--------+
2021-03-07|Quebec| 707|
+----------+------+--------+

### 2. Climat

Les fichiers du climat au Canada sont par station meteorolohique. Donc, on a besoin d'un fichier avec les stations par province et par année.

Cet fichier dois être telechargé sur DBFS

In [0]:
# Getting acces to file that contains weather station par province
df_stations_tmp = spark.read.csv("/FileStore/tables/Covid_DB/Weather/Station_Inventory_EN.csv", header=True, inferSchema= True)
df_stations = df_stations_tmp.select("Province","Station ID")
df_stations = df_stations.withColumnRenamed("Station ID","ID")

# Provinces format 
Provinces_stations = [x.Province for x in df_stations.select("Province").distinct().collect()]
Provinces_tmp.sort()
Provinces_stations.sort()

On cree une dictionaire qui contienne les estations meteorologique qu'on veux.

In [0]:
# This dictonary contains all the weather station IDs for each province. Ho
# Dictinoary {Province: list of stationsID}
stationsIDparProvince = {}
for i, prov in enumerate(Provinces_stations):
    df_tmp = df_stations.filter(df_stations.Province == prov)
    list_tmp = [x.ID for x in df_tmp.select("ID").distinct().collect()]
    stationsIDparProvince[Provinces_tmp[i]] = list_tmp

Le dictionaire precedent a tous les estation meteorologiques par province. Mais, si on telecharge les fichiers pour tous les estation du Canada, il va prendre beaucoup de temps. Les cluster d'une compte gratuite dans Databricks n'est pas assez pour telecharger tous les fichier.

Donc, On a choisi les estation meteorologuiques des villes les plus peuplées par province

In [0]:
# Considering only meteo stations of most populated cities by province
stationsIDparProvince = {'New Brunswick': [6206, 6207, 50390, 54282, 6157, 71700, 10981, 71668, 6159, 6248, 6249, 6250, 71609], #13
                         'Newfoundland and Labrador':  [6718, 6719, 50089, 6720, 6721, 6722, 48871, 27115, 6933, 6643], #10 
                         'Nunavut':[1758, 47388, 42503, 71909, 1720, 47427],  #6
                         'Quebec':[8343, 5414, 5415, 51157, 26855, 5420, 5418, 30165, 20719, 53001, 5590, 8375, 5249, 
                                   5250, 51457, 5251, 5528, 5529, 5292, 52138], #20  
                         'Prince Edward Island':[6525, 6526, 6527, 6528, 10800, 6546], #6
                         'Yukon':[1616, 1617, 50842, 1618, 48168, 26988,  10194],  #7
                         'British Columbia':[819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 108, 109, 110, 111, 112], # 15
                         'Saskatchewan':[3327, 3328, 46647, 46687, 3330, 3331, 3334, 3002, 3003, 3004, 3007, 3008, 3011, 3016], # 14
                         'Northwest Territories':[1706, 51058, 45467, 1707, 27338], #5
                         'Alberta':[45847, 2202, 2203, 2204, 45867, 2207, 2208, 2209, 52200, 1863, 1864, 1865, 31427, 
                                    30907, 1870, 1871, 1872, 43149, 2132, 2133], #20
                         'Nova Scotia':[6355, 6354, 6355, 49128, 43405, 43124, 43403, 50620, 6358, 6485, 6486, 6490, 6491], #13
                         'Manitoba':[3689, 3690, 3691, 3692, 3693, 27525, 3703, 3704, 3705, 28051, 3471, 3472, 3473], #13
                         'Ontario':[5051, 41863, 5052, 5053, 5054, 5055, 5056, 54239, 53678, 43203, 5065, 5066, 5067, 5068,
                                    4328, 4329, 4330, 30578, 4339, 4340, 4931, 4932, 4933, 52742, 4937]     #25    
}

Pour telecharger les fichier de chaque estation meteorologique à DBFS, on va a ecrire les address url pour chaque estation (par année) dans une dictionaire.

In [0]:
# Dictinary {Province: {year: {stationID.csv: url}}} 
# There are 2 province keys: For example, Quebec, and Quebec2021
URLparProvince = {}
dic_tmp = {}
years = [2020,2021]
for k,v in stationsIDparProvince.items():  
  year_dic = {}
  for y in years:
    dic_tmp = {}
    for stationID in v:
      filename = str(stationID)+".csv"
      url = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID="+str(stationID)+"&Year="+str(y)+"&Month=1&Day=14&timeframe=2"
      dic_tmp[filename] = url
    year_dic[y] = dic_tmp
  URLparProvince[k]=year_dic

In [0]:
#print(URLparProvince)

Chaque fichier du climat a plusieurs attributs. On a choisi les plus importants.

In [0]:
features = ["Date/Time", "Max Temp (°C)", "Min Temp (°C)", "Mean Temp (°C)", "Heat Deg Days (°C)", "Cool Deg Days (°C)", "Total Rain (mm)", "Total Precip (mm)", "Snow on Grnd (cm)", "Dir of Max Gust (10s deg)", "Spd of Max Gust (km/h)"]
cols = ["Max Temp (°C)", "Min Temp (°C)", "Mean Temp (°C)", "Heat Deg Days (°C)", "Cool Deg Days (°C)", "Total Rain (mm)", "Total Precip (mm)", "Snow on Grnd (cm)", "Dir of Max Gust (10s deg)", "Spd of Max Gust (km/h)"]

la classe suivante a des méthodes pourregrouper chaque fichier de station météo par date et province, et stocker la dataframe groupée dans DBFS

In [0]:
import datetime 
from pyspark.sql.functions import col
import functools 
from pyspark.sql.functions import lit

# number of days during 2021 in order to take only the line we are interested in
delta = (datetime.datetime.now() - datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")).days

class Handle_DFs:
  def __init__(self, list_dfs, province):
    self.list_dfs = list_dfs
    self.province = province
    
  # funtion to merge dataframes
  def unionAll_dfs(self):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), self.list_dfs)

  # function to group by date: all the features must be coverted to floats.
  def groupByDate_floats(self, features):
    df_tmp = self.unionAll_dfs()
    for col_name in features:
      df_tmp = df_tmp.withColumn(col_name, col(col_name).cast('float')) # convert to float
      df_out = df_tmp.groupBy("Date/Time").avg().orderBy("Date/Time") # groupBy Date
      df_out = df_out.withColumn("Province", lit(self.province)) # Include Province
    return df_out;
  
  # Saving dataframe into DBFS
  def SaveDF_tofile(self, df,filename):
    path_toSave_Results = filename +self.province
    df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save(path_toSave_Results)
    
  # Read DF from file or downloaded results
  def ReadDF_fromfile(self, update, filename, features):
    if(update):
      df_tmp = self.groupByDate_floats(features)
      self.SaveDF_tofile(df_tmp, filename)
    else:
      df_tmp = spark.read.csv(filename + self.province, header=True, inferSchema= True)  
    return df_tmp

In [0]:
# Remove previous data if necessary
main_path = "/FileStore/tables/Covid_DB_08Mar/Weather/"
path_to_save = main_path + "Results/"

# Deleting previous downloaded data:
if(update_2020weatherdata):
  dbutils.fs.rm(main_path + "2020", True)
  dbutils.fs.rm(path_to_save + "R2020/", True)
if(update_2021weatherdata):
  dbutils.fs.rm(main_path + "2021", True)
  dbutils.fs.rm(path_to_save + "R2021/", True)
if(update_2020weatherdata or update_2021weatherdata):
  dbutils.fs.rm(path_to_save + "RTotal/", True)

Téléchargement (ou lecture) de toutes les données météorologiques. Regroupez-les par date et par province. Stockage des données dans un DF.

Comme resultat, on a une liste des conditions météorologiques DF par Province

In [0]:
# For loop to built weather DB
df_weather_list = []
for k,year in URLparProvince.items():
  print("Province: ", k, " started: ",   datetime.datetime.now())
  list_df_2020 = []
  list_df_2021 = []
  if(update_2020weatherdata or update_2021weatherdata):
    # -----------
    for y,files in year.items():      
      path = main_path +str(y)+"/"
      for filename, url in files.items():
        #print("              3. KEY ", filename)
        if(y == 2021 and update_2021weatherdata):
          df_tmp = Handle_datafiles(url, path, filename).SaveAndReadFromUrl(delta)
          list_df_2021.append(df_tmp.select(features))
        elif(y == 2020 and update_2020weatherdata):
          df_tmp = Handle_datafiles(url, path, filename).SaveAndReadFromUrl()
          list_df_2020.append(df_tmp.select(features))
    # -----------
    
    df_2020 = Handle_DFs(list_df_2020, k).ReadDF_fromfile(update_2020weatherdata, path_to_save + "R2020/", cols)
    df_2021 = Handle_DFs(list_df_2021, k).ReadDF_fromfile(update_2021weatherdata, path_to_save + "R2021/", cols)
    
    df_weather_byProv = Handle_DFs([df_2020,df_2021], k).unionAll_dfs() 
    Handle_DFs(df_weather_byProv, k).SaveDF_tofile(df_weather_byProv, path_to_save + "RTotal/")          
  else:     
    df_weather_byProv = spark.read.csv(path_to_save+"RTotal/"+k, header=True, inferSchema= True)
    
  df_weather_list.append(df_weather_byProv)    
  print(k, "  ==  Stored in file and in List == ",   datetime.datetime.now())    

Province: New Brunswick started: 2021-03-08 23:43:03.223476
New Brunswick == Stored in file and in List == 2021-03-08 23:43:52.003626
Province: Newfoundland and Labrador started: 2021-03-08 23:43:52.003719
Newfoundland and Labrador == Stored in file and in List == 2021-03-08 23:44:28.097301
Province: Nunavut started: 2021-03-08 23:44:28.097643
Nunavut == Stored in file and in List == 2021-03-08 23:44:52.235111
Province: Quebec started: 2021-03-08 23:44:52.235215
Quebec == Stored in file and in List == 2021-03-08 23:45:59.498542
Province: Prince Edward Island started: 2021-03-08 23:45:59.498695
Prince Edward Island == Stored in file and in List == 2021-03-08 23:46:24.473405
Province: Yukon started: 2021-03-08 23:46:24.473512
Yukon == Stored in file and in List == 2021-03-08 23:46:52.358021
Province: British Columbia started: 2021-03-08 23:46:52.358119
British Columbia == Stored in file and in List == 2021-03-08 23:47:39.255800
Province: Saskatchewan started: 2021-03-08 23:47:39.255951
Saskatchewan == Stored in file and in List == 2021-03-08 23:48:20.966226
Province: Northwest Territories started: 2021-03-08 23:48:20.966353
Northwest Territories == Stored in file and in List == 2021-03-08 23:48:40.749044
Province: Alberta started: 2021-03-08 23:48:40.749143
Alberta == Stored in file and in List == 2021-03-08 23:49:40.636676
Province: Nova Scotia started: 2021-03-08 23:49:40.637077
Nova Scotia == Stored in file and in List == 2021-03-08 23:50:20.813140
Province: Manitoba started: 2021-03-08 23:50:20.813240
Manitoba == Stored in file and in List == 2021-03-08 23:51:02.690881
Province: Ontario started: 2021-03-08 23:51:02.690983
Ontario == Stored in file and in List == 2021-03-08 23:52:16.816411

On fusion toutes les DFs afin d'obtenir une seule DF du météo

In [0]:
df_weather = Handle_DFs(df_weather_list, k).unionAll_dfs() 

old_colnames = df_weather.columns
new_colnames = ["Date","MaxTemp", "MinTemp", "MeanTemp", "HeatDegDays", "CoolDegDays", "TotalRain", "TotalPrecip", "SnowOnGrnd", "DirOfMaxGust", "SpdOfMaxGust", "Province"]

for i,j in enumerate(old_colnames):
  df_weather = df_weather.withColumnRenamed(j,new_colnames[i])
  
df_weather.show()

+----------+-------------------+--------------------+-------------------+------------------+-----------+---------+-----------+----------+------------+------------+-------------+
 Date| MaxTemp| MinTemp| MeanTemp| HeatDegDays|CoolDegDays|TotalRain|TotalPrecip|SnowOnGrnd|DirOfMaxGust|SpdOfMaxGust| Province|
+----------+-------------------+--------------------+-------------------+------------------+-----------+---------+-----------+----------+------------+------------+-------------+
2020-01-01| 3.5| -3.200000047683716|0.20000000298023224|17.799999237060547| 0.0| null| null| null| 26.0| 40.0|New Brunswick|
2020-01-02| 1.5| -3.200000047683716|-0.8999999761581421|18.899999618530273| 0.0| null| null| null| 26.0| 53.0|New Brunswick|
2020-01-03| 3.799999952316284| -2.5999999046325684| 0.6000000238418579|17.399999618530273| 0.0| null| null| null| null| null|New Brunswick|
2020-01-04| 3.700000047683716| -2.200000047683716| 0.800000011920929|17.200000762939453| 0.0| null| null| null| null| null|New Brunswick|
2020-01-05|-1.2000000476837158| -6.800000190734863| -4.0| 22.0| 0.0| null| null| null| 32.0| 63.0|New Brunswick|
2020-01-06| -6.099999904632568| -8.199999809265137| -7.199999809265137|25.200000762939453| 0.0| null| null| null| 31.0| 31.0|New Brunswick|
2020-01-07| -2.200000047683716| -10.399999618530273| -6.300000190734863|24.299999237060547| 0.0| null| null| null| null| null|New Brunswick|
2020-01-08|-2.9000000953674316| -10.300000190734863| -6.599999904632568|24.600000381469727| 0.0| null| null| null| 29.0| 34.0|New Brunswick|
2020-01-09| -6.099999904632568| -14.699999809265137|-10.399999618530273|28.399999618530273| 0.0| null| null| null| 29.0| 50.0|New Brunswick|
2020-01-10| 2.5999999046325684| -15.800000190734863| -6.599999904632568|24.600000381469727| 0.0| null| null| null| 16.0| 37.0|New Brunswick|
2020-01-11| 10.899999618530273|-0.10000000149011612| 5.400000095367432|12.600000381469727| 0.0| null| null| null| 23.0| 64.0|New Brunswick|
2020-01-12| 1.2999999523162842| -8.600000381469727| -3.700000047683716|21.700000762939453| 0.0| null| null| null| 4.0| 47.0|New Brunswick|
2020-01-13| -6.300000190734863| -10.300000190734863| -8.300000190734863|26.299999237060547| 0.0| null| null| null| null| null|New Brunswick|
2020-01-14| -4.699999809265137| -7.800000190734863| -6.300000190734863|24.299999237060547| 0.0| null| null| null| null| null|New Brunswick|
2020-01-15|-2.0999999046325684| -6.800000190734863| -4.5| 22.5| 0.0| null| null| null| null| null|New Brunswick|
2020-01-16|-3.9000000953674316| -9.899999618530273| -6.900000095367432|24.899999618530273| 0.0| null| null| null| 34.0| 54.0|New Brunswick|
2020-01-17| -8.899999618530273| -19.600000381469727|-14.300000190734863| 32.29999923706055| 0.0| null| null| null| 33.0| 60.0|New Brunswick|
2020-01-18|-13.699999809265137| -21.299999237060547| -17.5| 35.5| 0.0| null| null| null| 25.0| 33.0|New Brunswick|
2020-01-19| -4.099999904632568| -15.199999809265137| -9.699999809265137|27.700000762939453| 0.0| null| null| null| 4.0| 39.0|New Brunswick|
2020-01-20| -4.5| -15.800000190734863|-10.199999809265137|28.200000762939453| 0.0| null| null| null| null| null|New Brunswick|
+----------+-------------------+--------------------+-------------------+------------------+-----------+---------+-----------+----------+------------+------------+-------------+
only showing top 20 rows

In [0]:
df_weather.createOrReplaceTempView("table_weather")
query_latest_rec = """SELECT * FROM table_weather """
latest_rec = sqlContext.sql(query_latest_rec)

### 3. Base de données de Population

Le fichier doit être télécharché à DBFS

In [0]:
df_population_data = spark.read.csv("/FileStore/tables/Covid_DB/Population/population.csv", header=True, inferSchema= True)

In [0]:
from pyspark.sql.functions import when

df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "NS", "Nova Scotia").otherwise(df_population_data["ProvinceAvr"]))
df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "NL", "Newfoundland and Labrador").otherwise(df_population_data["ProvinceAvr"]))
df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "NT", "Northwest Territories").otherwise(df_population_data["ProvinceAvr"]))
df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "QC", "Quebec").otherwise(df_population_data["ProvinceAvr"]))
df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "BC", "British Columbia").otherwise(df_population_data["ProvinceAvr"]))
df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "MB", "Manitoba").otherwise(df_population_data["ProvinceAvr"]))
df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "NU", "Nunavut").otherwise(df_population_data["ProvinceAvr"]))
df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "ON", "Ontario").otherwise(df_population_data["ProvinceAvr"]))
df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "SK", "Saskatchewan").otherwise(df_population_data["ProvinceAvr"]))
df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "AB", "Alberta").otherwise(df_population_data["ProvinceAvr"]))
df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "PE", "Prince Edward Island").otherwise(df_population_data["ProvinceAvr"]))
df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "YT", "Yukon").otherwise(df_population_data["ProvinceAvr"]))
df_population_data = df_population_data.withColumn("ProvinceAvr", when(df_population_data["ProvinceAvr"] == "NB", "New Brunswick").otherwise(df_population_data["ProvinceAvr"]))

In [0]:
old_colnames = df_population_data.columns
new_colnames = ['Province', 'TousAges', 'ans0_4', 'ans5_9', 'ans10_14', 'ans15_19', 'ans20_24', 'ans25_29', 'ans30_34', 'ans35_39', 'ans40_44', 'ans45_49', 'ans50_54', 'ans55_59', 'ans60_64', 'ans65_69', 'ans70_74', 'ans75_79', 'ans80_84', 'ans85_89', 'ans90_94', 'ans95_99',  'ans100etplus', 'AgeMedian']

for i,j in enumerate(old_colnames):
  df_population_data = df_population_data.withColumnRenamed(j,new_colnames[i])
  
df_population_data.show()

+--------------------+--------+------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+---------+
 Province|TousAges|ans0_4|ans5_9|ans10_14|ans15_19|ans20_24|ans25_29|ans30_34|ans35_39|ans40_44|ans45_49|ans50_54|ans55_59|ans60_64|ans65_69|ans70_74|ans75_79|ans80_84|ans85_89|ans90_94|ans95_99|ans100etplus|AgeMedian|
+--------------------+--------+------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+---------+
Newfoundland and ...| 522103| 20402| 23376| 26186| 26994| 28910| 28076| 28942| 30151| 31426| 35754| 40214| 43105| 42339| 39718| 33179| 20458| 12617| 6662| 2715| 751| 128| 47.4|
 Ontario|14734014|723016|762654| 792947| 852405| 1039661| 1077433| 1041952| 992844| 921378| 932058| 968546| 1073519| 961243| 803962| 673546| 461015| 319548| 204227| 98638| 29527| 3895| 40.4|
 Manitoba| 1379263| 85220| 89507| 86583| 85808| 96636| 97157| 98789| 95203| 86739| 82541| 80302| 89949| 83163| 70981| 57001| 38329| 25922| 16952| 8968| 3010| 503| 37.6|
 Quebec| 8574571|430250|465890| 457420| 426496| 499370| 564669| 548066| 578273| 581724| 527233| 546847| 637724| 619126| 529279| 440008| 313752| 198497| 128147| 60879| 18086| 2835| 42.7|
 New Brunswick| 781476| 33582| 38199| 40354| 40662| 43490| 43981| 44703| 46655| 48179| 51551| 54044| 63300| 61514| 55477| 46962| 30462| 19475| 11705| 5213| 1704| 264| 46.1|
 Yukon| 42052| 2413| 2508| 2246| 2067| 2346| 2828| 3468| 3549| 3050| 2864| 2753| 3233| 3116| 2385| 1532| 862| 464| 246| 84| 31| 7| 39.4|
Northwest Territo...| 45161| 2963| 3081| 3010| 2793| 3028| 3688| 3705| 3405| 3348| 2919| 3238| 3389| 2619| 1818| 1048| 603| 293| 133| 57| 21| 2| 35.5|
 Nova Scotia| 979351| 42443| 46177| 48170| 50776| 61556| 63359| 61264| 58683| 56907| 61194| 65276| 78780| 75941| 66676| 57399| 37734| 24225| 14184| 6318| 1930| 359| 45.0|
 Saskatchewan| 1178681| 75200| 78696| 76730| 71304| 75402| 77488| 85751| 86664| 76067| 67397| 65716| 77024| 74222| 62071| 46528| 31910| 22791| 15931| 8492| 2773| 524| 37.8|
Prince Edward Island| 159625| 7199| 8248| 8966| 9340| 11577| 10321| 9212| 9314| 9586| 10231| 10319| 12005| 11350| 10116| 9106| 5589| 3641| 2244| 935| 281| 45| 42.9|
 Alberta| 4421876|269163|277785| 276328| 256742| 277328| 314507| 356226| 359301| 319889| 288546| 266489| 284259| 264339| 210073| 157657| 102977| 68565| 44033| 20921| 5746| 1002| 37.5|
 Nunavut| 39353| 4196| 4276| 3987| 3283| 3173| 3276| 3179| 2767| 2334| 2154| 2277| 1761| 1119| 698| 459| 235| 114| 41| 18| 3| 3| 26.2|
 British Columbia| 5147712|225897|244206| 249173| 272195| 340325| 358457| 376466| 363871| 323620| 325674| 343894| 376848| 360150| 314021| 262197| 174377| 115218| 73205| 35355| 10613| 1950| 42.2|
+--------------------+--------+------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+---------+

In [0]:
df_population_data.createOrReplaceTempView("table_population")
query_latest_rec = """SELECT * FROM table_population"""
latest_rec = sqlContext.sql(query_latest_rec)
#latest_rec.show()

### Integration de données

#### - Integration de la base de données du Covid et Population

In [0]:
df_covid.createOrReplaceTempView("table_covid")
df_population_data.createOrReplaceTempView("table_population")
query_latest_rec = """SELECT tc.pruid, tc.prname, tc.date, tc.numtotal, tc.numtoday, tc.numprob, tc.numdeaths, tc.numtotal, tc.numtested, tc.numrecover, tc.percentrecover, tc.ratetested, tc.percentoday, tc.ratetotal, tc.ratedeaths, tc.numdeathstoday, tc.percentdeath, tc.numtestedtoday, tc.numrecoveredtoday, tc.percentactive,  tc.numactive, tc.rateactive,  tc.numtotal_last14, tc.ratetotal_last14, tc.numdeaths_last14, tc.ratedeaths_last14, tc.numtotal_last7, tc.ratetotal_last7, tc.numdeaths_last7, tc.ratedeaths_last7, tc.avgtotal_last7, tc.avgincidence_last7, tc.avgdeaths_last7, tc.avgratedeaths_last7, tp.TousAges, tp.ans0_4, tp.ans5_9, tp.ans10_14, tp.ans15_19, tp.ans20_24, tp.ans25_29, tp.ans30_34, tp.ans35_39, tp.ans40_44, tp.ans45_49, tp.ans50_54, tp.ans55_59, tp.ans60_64, tp.ans65_69, tp.ans70_74, tp.ans75_79, tp.ans80_84, tp.ans85_89, tp.ans90_94, tp.ans95_99, tp.ans100etplus, tp.AgeMedian
  FROM table_population tp  FULL JOIN table_covid tc
    ON tp.Province = tc.prname"""
df_covid_population = sqlContext.sql(query_latest_rec)
df_covid_population.show(2)

+-----+-------+----------+--------+--------+-------+---------+--------+---------+----------+--------------+----------+-----------+---------+----------+--------------+------------+--------------+-----------------+-------------+---------+----------+---------------+----------------+----------------+-----------------+--------------+---------------+---------------+----------------+--------------+------------------+---------------+-------------------+--------+------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+---------+
pruid| prname| date|numtotal|numtoday|numprob|numdeaths|numtotal|numtested|numrecover|percentrecover|ratetested|percentoday|ratetotal|ratedeaths|numdeathstoday|percentdeath|numtestedtoday|numrecoveredtoday|percentactive|numactive|rateactive|numtotal_last14|ratetotal_last14|numdeaths_last14|ratedeaths_last14|numtotal_last7|ratetotal_last7|numdeaths_last7|ratedeaths_last7|avgtotal_last7|avgincidence_last7|avgdeaths_last7|avgratedeaths_last7|TousAges|ans0_4|ans5_9|ans10_14|ans15_19|ans20_24|ans25_29|ans30_34|ans35_39|ans40_44|ans45_49|ans50_54|ans55_59|ans60_64|ans65_69|ans70_74|ans75_79|ans80_84|ans85_89|ans90_94|ans95_99|ans100etplus|AgeMedian|
+-----+-------+----------+--------+--------+-------+---------+--------+---------+----------+--------------+----------+-----------+---------+----------+--------------+------------+--------------+-----------------+-------------+---------+----------+---------------+----------------+----------------+-----------------+--------------+---------------+---------------+----------------+--------------+------------------+---------------+-------------------+--------+------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+---------+
 48|Alberta|2020-03-08| 1| 1| 0| 0| 1| null| null| null| null| 100.0| 0.02| 0.0| 0.0| 0.0| null| null| 100.0| 1| 0.02| null| null| null| null| null| null| null| null| null| null| null| null| 4421876|269163|277785| 276328| 256742| 277328| 314507| 356226| 359301| 319889| 288546| 266489| 284259| 264339| 210073| 157657| 102977| 68565| 44033| 20921| 5746| 1002| 37.5|
 48|Alberta|2020-03-09| 7| 6| 0| 0| 7| null| null| null| null| 600.0| 0.16| 0.0| 0.0| 0.0| null| null| 100.0| 7| 0.16| null| null| null| null| null| null| null| null| null| null| null| null| 4421876|269163|277785| 276328| 256742| 277328| 314507| 356226| 359301| 319889| 288546| 266489| 284259| 264339| 210073| 157657| 102977| 68565| 44033| 20921| 5746| 1002| 37.5|
+-----+-------+----------+--------+--------+-------+---------+--------+---------+----------+--------------+----------+-----------+---------+----------+--------------+------------+--------------+-----------------+-------------+---------+----------+---------------+----------------+----------------+-----------------+--------------+---------------+---------------+----------------+--------------+------------------+---------------+-------------------+--------+------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+---------+
only showing top 2 rows

#### - Integration de la base de données du (Covid+Population) et Climat

In [0]:
df_covid_population.createOrReplaceTempView("table_covid_population")
query_latest_rec = """SELECT tcp.pruid, tcp.prname, tcp.date, tcp.numtoday, tcp.numprob, tcp.numdeaths, tcp.numtotal, tcp.numtested, tcp.numrecover, tcp.percentrecover, tcp.ratetested, tcp.percentoday, tcp.ratetotal, tcp.ratedeaths, tcp.numdeathstoday, tcp.percentdeath, tcp.numtestedtoday, tcp.numrecoveredtoday, tcp.percentactive,  tcp.numactive, tcp.rateactive,  tcp.numtotal_last14, tcp.ratetotal_last14, tcp.numdeaths_last14, tcp.ratedeaths_last14, tcp.numtotal_last7, tcp.ratetotal_last7, tcp.numdeaths_last7, tcp.ratedeaths_last7, tcp.avgtotal_last7, tcp.avgincidence_last7, tcp.avgdeaths_last7, tcp.avgratedeaths_last7, tcp.TousAges, tcp.ans0_4, tcp.ans5_9, tcp.ans10_14, tcp.ans15_19, tcp.ans20_24, tcp.ans25_29, tcp.ans30_34, tcp.ans35_39, tcp.ans40_44, tcp.ans45_49, tcp.ans50_54, tcp.ans55_59, tcp.ans60_64, tcp.ans65_69, tcp.ans70_74, tcp.ans75_79, tcp.ans80_84, tcp.ans85_89, tcp.ans90_94, tcp.ans95_99, tcp.ans100etplus, tcp.AgeMedian, tw.MaxTemp, tw.MinTemp, tw.MeanTemp, tw.HeatDegDays, tw.CoolDegDays, tw.TotalRain, tw.TotalPrecip, tw.SnowOnGrnd, tw.DirOfMaxGust, tw.SpdOfMaxGust
FROM table_covid_population tcp
INNER JOIN  table_weather tw 
ON tcp.date = tw.Date AND tcp.prname = tw.Province
"""

df_COVID = sqlContext.sql(query_latest_rec)
df_COVID.show(2)

+-----+----------------+----------+--------+-------+---------+--------+---------+----------+--------------+----------+-----------+---------+----------+--------------+------------+--------------+-----------------+-------------+---------+----------+---------------+----------------+----------------+-----------------+--------------+---------------+---------------+----------------+--------------+------------------+---------------+-------------------+--------+------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+---------+-------------------+-------------------+------------------+------------------+-----------+---------+-----------+----------+------------+------------+
pruid| prname| date|numtoday|numprob|numdeaths|numtotal|numtested|numrecover|percentrecover|ratetested|percentoday|ratetotal|ratedeaths|numdeathstoday|percentdeath|numtestedtoday|numrecoveredtoday|percentactive|numactive|rateactive|numtotal_last14|ratetotal_last14|numdeaths_last14|ratedeaths_last14|numtotal_last7|ratetotal_last7|numdeaths_last7|ratedeaths_last7|avgtotal_last7|avgincidence_last7|avgdeaths_last7|avgratedeaths_last7|TousAges|ans0_4|ans5_9|ans10_14|ans15_19|ans20_24|ans25_29|ans30_34|ans35_39|ans40_44|ans45_49|ans50_54|ans55_59|ans60_64|ans65_69|ans70_74|ans75_79|ans80_84|ans85_89|ans90_94|ans95_99|ans100etplus|AgeMedian| MaxTemp| MinTemp| MeanTemp| HeatDegDays|CoolDegDays|TotalRain|TotalPrecip|SnowOnGrnd|DirOfMaxGust|SpdOfMaxGust|
+-----+----------------+----------+--------+-------+---------+--------+---------+----------+--------------+----------+-----------+---------+----------+--------------+------------+--------------+-----------------+-------------+---------+----------+---------------+----------------+----------------+-----------------+--------------+---------------+---------------+----------------+--------------+------------------+---------------+-------------------+--------+------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+---------+-------------------+-------------------+------------------+------------------+-----------+---------+-----------+----------+------------+------------+
 35| Ontario|2020-01-31| 3| 0| 0| 3| null| null| null| null| 300.0| 0.02| 0.0| 0.0| 0.0| null| null| 100.0| 3| 0.02| null| null| null| null| null| null| null| null| null| null| null| null|14734014|723016|762654| 792947| 852405| 1039661| 1077433| 1041952| 992844| 921378| 932058| 968546| 1073519| 961243| 803962| 673546| 461015| 319548| 204227| 98638| 29527| 3895| 40.4|-3.1999999582767487|-10.400000095367432|-6.800000071525574|24.799999237060547| 0.0| null| 0.0| 25.0| null| null|
 59|British Columbia|2020-01-31| 1| 0| 0| 1| null| null| null| null| 100.0| 0.02| 0.0| 0.0| 0.0| null| null| 100.0| 1| 0.02| null| null| null| null| null| null| null| null| null| null| null| null| 5147712|225897|244206| 249173| 272195| 340325| 358457| 376466| 363871| 323620| 325674| 343894| 376848| 360150| 314021| 262197| 174377| 115218| 73205| 35355| 10613| 1950| 42.2| 6.0| -3.0| 1.5| 16.5| 0.0| 163.5| 163.5| 0.0| null| null|
+-----+----------------+----------+--------+-------+---------+--------+---------+----------+--------------+----------+-----------+---------+----------+--------------+------------+--------------+-----------------+-------------+---------+----------+---------------+----------------+----------------+-----------------+--------------+---------------+---------------+----------------+--------------+------------------+---------------+-------------------+--------+------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+---------+-------------------+-------------------+------------------+--

#### Sauvegarde le DF complet dans un fichier sur DBFS

In [0]:
dbutils.fs.rm("/FileStore/tables/Covid_DB_08Mar/Total_Result/", True)
path_toSave = "dbfs:/FileStore/tables/Covid_DB_08Mar/Total_Result/"
df_COVID.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save(path_toSave)

#### Sauvegarde la dernière ligne du DF dans un fichier à utiliser pour la prédiction

In [0]:
df_COVID.createOrReplaceTempView("table_prediction")
query_latest_rec = """SELECT * FROM table_prediction ORDER BY date DESC LIMIT 13"""
df_predict = sqlContext.sql(query_latest_rec)

dbutils.fs.rm("/FileStore/tables/Covid_DB_08Mar/ForPrediction/", True)
path_toSave = "dbfs:/FileStore/tables/Covid_DB_08Mar/ForPrediction/"
df_predict.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save(path_toSave)

In [0]:
query_province_id = """SELECT pruid, prname FROM table_prediction ORDER BY date DESC LIMIT 13"""
province_id = sqlContext.sql(query_province_id)
province_id.show()

+-----+--------------------+
pruid| prname|
+-----+--------------------+
 60| Yukon|
 59| British Columbia|
 48| Alberta|
 47| Saskatchewan|
 46| Manitoba|
 35| Ontario|
 24| Quebec|
 10|Newfoundland and ...|
 13| New Brunswick|
 12| Nova Scotia|
 11|Prince Edward Island|
 61|Northwest Territo...|
 62| Nunavut|
+-----+--------------------+